In [18]:
import os
from os.path import join
import numpy as np
import pandas as pd

sh_or_bat = "sh"
project_folder = r"/Users/fferegrino/Documents/GitHub/information-retrieval/ae2/q2/"
base_folder = r"/Users/fferegrino/Documents/GitHub/information-retrieval/ae2/q2/"
hw_files = r"/Users/fferegrino/Documents/ae1/"
terrier_path =      r"/Users/fferegrino/Documents/terrier-core-4.2"
if os.name == 'nt':
    terrier_path =      r"C:\Users\anton\Terrier"
    project_folder = r"C:\Users\anton\Documents\GitHub\information-retrieval\ae2\q2"
    base_folder = r"C:\Users\anton\Documents\GitHub\information-retrieval\ae2"
    hw_files = r"C:\terrier_data"
    sh_or_bat = "bat"
    
if not os.path.exists(project_folder):
    os.makedirs(project_folder)

logs_folder =       join(project_folder,"logs")
results_folder =    join(project_folder,"results")
features_file =     join(project_folder,"features.txt")
jforest_file =      join(project_folder,"jforests.properties")
terrier_index =     join(hw_files,r"indices/blocks_fields_stemming")
training_topics =   join(hw_files,r"topics/training/topics")
validation_topics = join(hw_files,r"topics/validation/topics")
training_qrels =    join(hw_files,r"topics/training/qrels")
validation_qrels =  join(hw_files,r"topics/validation/qrels")
hp04_topics =       join(hw_files,r"topics/HP04/topics")
hp04_qrels =        join(hw_files,r"topics/HP04/qrels")
jar_file =          join(base_folder, "search-features\target\ircourse-1.0-SNAPSHOT.jar")

terrier = join(terrier_path, "bin", "trec_terrier.%s" % sh_or_bat)
teval = join(terrier_path, "bin", "trec_eval.%s" % sh_or_bat)
anyclass = join(terrier_path, "bin", "anyclass.%s" % sh_or_bat)
letor_tr_file= join(results_folder, "tr.letor")
letor_va_file= join(results_folder, "va.letor")
ensemble_file= join(results_folder, "ensemble.txt")

def write_log(file, content, command=None):
    with open(join(logs_folder, file), "w") as w:
        if command is not None:
            w.write(command + "\n\n")
        for line in content:
            w.write(line + "\n")

## Q2:  
Now, you should implement two additional proximity search features – a proximity search feature allows documents where the query terms occur closely together to be boosted. We require
that you implement two of the functions numbered 1-5 in the following paper: 
 > Ronan Cummins and Colm O'Riordan. 2009. Learning in a pairwise term-term proximity framework for information retrieval. In Proceedings of ACM SIGIR 2009. http://ir.dcs.gla.ac.uk/~ronanc/papers/cumminsSIGIR09.pdf
 
NB: You should calculate your feature by aggregating the function score (mean or min or max, as appropriate) over all pairs of query terms.  

You will implement your new features as two DocumentScoreModifiers (DSM) classes, using the example DSM code provided in the Github repository(https://github.com/cmacdonald/IRcourseHM). 

You can add a DSM as an additional feature by appending its full name to the feature file, e.g.:  

```
DSM:org.myclass.MyProx1DSM  
```

### Q2a:  
Name the two proximity features you have chosen to implement and provide a brief rationale for your choice of these two particular features, especially in terms of how they might affect the performance of the deployed LTR baseline approach of Q1.

#### Features

| Feature       | Class                              |
| ------------- |:----------------------------------:|
| min_dist      | `uk.ac.gla.dcs.dsms.MinDistDsm`    |
| avg_min_dist  | `uk.ac.gla.dcs.dsms.AvgMinDistDsm` |


In [12]:
# Build package
build_log = !mvn -f search-features/pom.xml clean package

In [19]:
features_file = join(project_folder,"features_min_dist.txt")

letor_tr_file= join(results_folder, "tr_min_dist.letor")
results_file = join(results_folder,"pl2_tr_ltr_min_dist.res")
command="CLASSPATH=%s %s -r -Dtrec.topics=%s " % (jar_file, terrier, training_topics) + \
    "-Dtrec.results.file=%s -Dtrec.model=PL2 -Dterrier.index.path=%s " % (results_file,  terrier_index) + \
    "-Dtrec.matching=FatFeaturedScoringMatching,org.terrier.matching.daat.FatFull " + \
    "-Dtrec.querying.outputformat=Normalised2LETOROutputFormat " + \
    "-Dquerying.postprocesses.order=QueryExpansion,org.terrier.learning.LabelDecorator " + \
    "-Dquerying.postprocesses.controls=labels:org.terrier.learning.LabelDecorator,qe:QueryExpansion " + \
    "-Dquerying.default.controls=labels:on " + \
    "-Dlearning.labels.file=%s " % training_qrels + \
    "-Dtrec.results.file=%s -Dproximity.dependency.type=SD " % letor_tr_file + \
    "-Dfat.featured.scoring.matching.features=FILE -Dfat.featured.scoring.matching.features.file=%s " % features_file 
    
run_results = !$command
write_log("pl2_tr_ltr_min_dist_querying.log", run_results, command=command)


letor_va_file= join(results_folder, "va_min_dist.letor")
results_file = join(results_folder,"pl2_va_ltr_min_dist.res")
command="CLASSPATH=%s %s -r -Dtrec.topics=%s " % (jar_file, terrier, validation_topics) + \
    "-Dtrec.results.file=%s -Dtrec.model=PL2 -Dterrier.index.path=%s " % (results_file,  terrier_index) + \
    "-Dtrec.matching=FatFeaturedScoringMatching,org.terrier.matching.daat.FatFull " + \
    "-Dtrec.querying.outputformat=Normalised2LETOROutputFormat " + \
    "-Dquerying.postprocesses.order=QueryExpansion,org.terrier.learning.LabelDecorator " + \
    "-Dquerying.postprocesses.controls=labels:org.terrier.learning.LabelDecorator,qe:QueryExpansion " + \
    "-Dquerying.default.controls=labels:on " + \
    "-Dlearning.labels.file=%s " % validation_qrels + \
    "-Dtrec.results.file=%s -Dproximity.dependency.type=SD " % letor_va_file + \
    "-Dfat.featured.scoring.matching.features=FILE -Dfat.featured.scoring.matching.features.file=%s " % features_file 
    
run_results = !$command
write_log("pl2_va_ltr_min_dist_querying.log", run_results, command=command)


command="%s edu.uci.jforests.applications.Runner " % anyclass + \
    "--config-file %s " % jforest_file + \
    "--cmd=generate-bin --ranking --folder %s " % results_folder + \
    "--file tr.letor --file va.letor"
    
run_results = !$command
write_log("anyclass_run_min_dist_1.log", run_results, command=command)


tr_bin = join(results_folder, "tr_min_dist.bin")
va_bin = join(results_folder, "va_min_dist.bin")
command="%s edu.uci.jforests.applications.Runner " % anyclass + \
    "--config-file %s " % jforest_file + \
    "--cmd=train --ranking --folder %s " % results_folder + \
    "--train-file %s --validation-file %s " % (tr_bin, va_bin) + \
    "--output-model %s" % ensemble_file
    
run_results = !$command
write_log("anyclass_run_min_dist_2.log", run_results, command=command)


ltr_test_results_file = join(results_folder,"pl2_te_ltr_min_dist.res")
jforest_statistics = join(results_folder, "jforests-feature-stats.txt")
command="CLASSPATH=%s %s -r -Dtrec.model=PL2 -Dtrec.topics=%s " % (jar_file, terrier, hp04_topics) + \
    "-Dtrec.matching=JforestsModelMatching,FatFeaturedScoringMatching,org.terrier.matching.daat.FatFull " + \
    "-Dfat.featured.scoring.matching.features=FILE -Dfat.featured.scoring.matching.features.file=%s " % features_file + \
    "-Dtrec.results.file=%s  -Dterrier.index.path=%s " % (ltr_test_results_file,  terrier_index) + \
    "-Dfat.matching.learned.jforest.model=%s " % ensemble_file + \
    "-Dfat.matching.learned.jforest.statistics=%s " % jforest_statistics + \
    "-Dproximity.dependency.type=SD"
    
    
run_results = !$command
write_log("pl2_te_ltr_querying.log", run_results, command=command)


eval_file = join(results_folder, "pl2_ltr_min_dist.eval")
command = "%s %s %s -q > %s" % (teval, hp04_qrels, ltr_test_results_file, eval_file)
run_results = !$command
write_log("pl2_ltr_eval.log", run_results, command=command)